In [26]:
# Climate: Use Text Similarity (article spinning) to Connect News Media
import pandas as pd
import numpy as np
import os
import sys
import json
import re
import nltk

# Load the .npz file
file_path = '/Users/maxzhu/Downloads/37000_proj/ground_news_webscraper/pairwise_similarity_scores_climate_sample.npz' 
# sample 0.5% of the pairs
data = np.load(file_path)
# save results as a DataFrame
data = data['results']
df_results = pd.DataFrame(data)
print(df_results.head())
df_results = df_results[(df_results['title_sim'] > 0.6) & (df_results['article_sim'] > 0.7)]
df_results.shape


    i      j  title_sim  article_sim
0  24  17924        0.0     0.101125
1  24  18141        0.0     0.091712
2  24  18319        0.0     0.088305
3  24  18654        0.0     0.065082
4  24  18748        0.0     0.043199


(453, 4)

In [3]:
df = pd.read_csv('/Users/maxzhu/Downloads/37000_proj/ground_news_webscraper/climate-change_articles_en.csv')

In [27]:
import numpy as np
import pandas as pd

# Extract the relevant columns from the metadata DataFrame as numpy arrays.
story_ids = df['story_id'].values
source_factuality = df['source_factuality'].values
source_bias = df['source_bias'].values
source_owners = df['source_owners'].values

# Get the paired indices from df_results.
i_idx = df_results['i'].values
j_idx = df_results['j'].values

# Define a function to create a valid mask (exclude nan and "unknown" values).
def valid_mask(arr):
    # Using pd.isna to catch NaN values; compare against the string "unknown".
    valid_i = (~pd.isna(arr[i_idx])) & (arr[i_idx] != "unknown")
    valid_j = (~pd.isna(arr[j_idx])) & (arr[j_idx] != "unknown")
    return valid_i & valid_j

# Prepare the columns to examine.
columns = {
    'story_id': story_ids,
    'source_bias': source_bias,
    'source_owners': source_owners
}

print("General Matching Statistics (ignoring pairs with 'unknown' or NaN):")
for col_name, arr in columns.items():
    # Build the valid mask for the current column.
    valid = valid_mask(arr)
    valid_count = np.sum(valid)
    if valid_count > 0:
        # Calculate proportion of matches among the valid pairs.
        match_mask = (arr[i_idx] == arr[j_idx])
        prop = np.mean(match_mask[valid])
        print(f"  {col_name:20s}: {prop:.2%} matches over {valid_count} valid pairs")
    else:
        print(f"  {col_name:20s}: No valid pairs available.")
    
print("=" * 50)

# Now, for each examined column, print 3 specific examples of mismatches among valid pairs.
for col_name, arr in columns.items():
    print(f"\nExamples of mismatches in '{col_name}':")
    valid = valid_mask(arr)
    # Compute mismatches only for valid pairs.
    mismatch_mask = (arr[i_idx] != arr[j_idx]) & valid
    mismatch_indices = np.where(mismatch_mask)[0]
    
    if len(mismatch_indices) == 0:
        print("  No mismatches found among valid pairs.")
    else:
        num_examples = min(10, len(mismatch_indices))
        for count, idx in enumerate(mismatch_indices[:num_examples], start=1):
            i_val = i_idx[idx]
            j_val = j_idx[idx]
            print(f"  Example {count}:")
            print(f"    Article 1 {col_name}: {df.loc[i_val, col_name]}, {df.loc[i_val, 'article_title'], df.loc[i_val, 'source_name']}")
            print(f"    Article 2 {col_name}: {df.loc[j_val, col_name]}, {df.loc[j_val, 'article_title'], df.loc[j_val, 'source_name']}")


General Matching Statistics (ignoring pairs with 'unknown' or NaN):
  story_id            : 96.25% matches over 453 valid pairs
  source_bias         : 57.72% matches over 149 valid pairs
  source_owners       : 40.22% matches over 92 valid pairs

Examples of mismatches in 'story_id':
  Example 1:
    Article 1 story_id: 400dabdf-c81e-482a-9802-e8fd9e140b1b, ('Access to affordable energy for 300m Africans by 2030 doable - Presidential aide', 'QUICK NEWS AFRICA')
    Article 2 story_id: rss_46638_1738033811444_6, ('Access to affordable energy for 300m Africans by 2030 doable - Presidential aide', 'EnviroNews Nigeria -')
  Example 2:
    Article 1 story_id: 179771b1-4fb3-4fb9-a228-61fa7cc021c8, ('David Moyes returns to Everton as manager', 'Jersey Evening Post')
    Article 2 story_id: 5330e6c6-9962-4e3d-b936-9e929d923f8c, ('David Moyes returns as manager of struggling Everton', 'Barrie Today')
  Example 3:
    Article 1 story_id: 179771b1-4fb3-4fb9-a228-61fa7cc021c8, ('David Moyes retur

In [ ]:
# Create a directed graph G, with weights

import networkx as nx
G = nx.DiGraph()
for row in df_results.iterrows():
    i  = row[1]['i']
    j = row[1]['j']
    source_i = df['source_name'][i]
    source_j = df['source_name'][j]
    time_i = df['article_date_publish'][i]
    time_j = df['article_date_publish'][j]
    try:
        if source_i != source_j:
            if time_i < time_j:
                s= source_i
                e = source_j
            else:
                s = source_j
                e = source_i
            if G.has_edge(s, e):
                G[s][e]['weight'] += 1
            else:
                G.add_edge(s, e, weight=1)
    except Exception as e:
        print(f"Error {e} for {source_i} and {source_j}")

# check nodes in G by out_degree
out_degree = G.out_degree()
out_degree = dict(out_degree)
out_degree = {k: v for k, v in sorted(out_degree.items(), key=lambda item: item[1], reverse=True)[:20]}
print(out_degree)


Error '<' not supported between instances of 'str' and 'float' for The Star Kuala Lumpur and Space Daily
Error '<' not supported between instances of 'str' and 'float' for The Straits Times and risingnepaldaily.com
Error '<' not supported between instances of 'str' and 'float' for eNCA and winddaily.com
Error '<' not supported between instances of 'str' and 'float' for Malay Mail and Space Daily
Error '<' not supported between instances of 'str' and 'float' for Legit.ng - Nigeria news. and oilgasdaily.com
Error '<' not supported between instances of 'str' and 'float' for CoreTV News and Nigerian News. Latest Nigeria News. Your online Nigerian Newspaper.
Error '<' not supported between instances of 'str' and 'float' for The Straits Times and Energy News for the United States Oil & Gas Industry | EnergyNow.com
Error '<' not supported between instances of 'str' and 'float' for Punch Newspapers and Nigerian News. Latest Nigeria News. Your online Nigerian Newspaper.
Error '<' not supported 

In [31]:
# check nodes in G by out_degree
in_degree = G.in_degree()
in_degree = dict(in_degree)
in_degree = {k: v for k, v in sorted(in_degree.items(), key=lambda item: item[1], reverse=True)[:20]}
print(in_degree)

{'Barrie Today': 10, 'Rocky Mountain Outlook': 8, 'Midland Today': 8, 'Newmarket Today': 7, 'KCRA 3': 7, 'Victoria News': 7, 'Bradford Today': 6, 'Newsday': 6, 'Koco News5': 5, 'haltonhillstoday.ca': 5, 'thealbertan.com': 5, 'Sicamous Eagle Valley News': 5, 'Associated Press News': 5, 'Winnipeg Sun': 5, 'Kitimat Northern Sentinel': 5, 'Moose Jaw Today': 5, 'Orillia Matters': 4, 'Collingwood Today': 4, 'Sask Today': 4, 'Coast Reporter': 4}


In [30]:
# find all neighbors of a node
neighbors = list(G.neighbors('haltonhillstoday.ca'))


['Winnipeg Free Press',
 'Collingwood Today',
 'Orillia Matters',
 'Bowen Island Undercurrent',
 '1059theregion.com',
 'Bradford Today',
 'Barrie Today',
 'The NOTL Local',
 'fitzhugh.ca',
 'SooToday.com',
 'Newmarket Today',
 'Oakville News']

In [32]:
list(G.neighbors('Victoria News'))

['crestonvalleyadvance.ca']